<a href="https://colab.research.google.com/github/pela-andrea/SoulCode_Academy_ADS2023/blob/main/09_03_MongoDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset AirBnB: MongoDb + Pandas
##Projeto realizado no bootcamp de Análise de Dados/Martech da SoulCode Academy em Março de 2023.


## - Criar 3 bancos de dados com os arquivos utilizados previamente e enviá-los ao MongoDb.

##- Recuperá-los do Mongo via python e transformá-los em df.

##- Tratamento dos dados.

##- Análise.

##**Preparar as Bibliotecas**


In [ ]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pymongo

from pymongo import MongoClient

**Conector MongoAtlas**

In [ ]:
uri = "mongodb+srv://pelaandrea.ipcl7g8.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri, tls=True, tlsCertificateKeyFile='/content/X509-cert-1821336394932634981.pem',
                     )

db = client['soulcode']
colecao = db['tarefa']


In [ ]:
doc_count = colecao.count_documents({})
print(doc_count)

0


##Integração com Pandas e Google

In [ ]:
import pandas as pd
import numpy as np
from google.cloud import storage
import os

##Vinculando o GoogleCloud

In [ ]:
serviceAccount = '/content/ivory-amphora-377819-2df80ceb77e9.json' #Carregar o Tokien da conta de serviço, para quando chegar lá no Storage ele já chegue autorizado
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client() #comando da biblioteca cloud storage google
bucket = client.get_bucket('pela_andrea01') #cliente que obtem o bucket (nome do bucket)
bucket.blob('ocorrenciass.csv') #nome do arquivo que será manipulado
path = 'gs://pela_andrea01/brutos/ocorrenciass.csv' #caminho do arquivo, pegar sempre o link do Gsuit para ir direto pelo google

#1º Df - Cenipas - ocorrencias.*csv*

##Incluindo o arquivo CSV

In [ ]:
path = '/content/ocorrenciass.csv'

In [ ]:
## Ler o arquivo csv
df1 = pd.read_csv(path,sep=';',encoding='ISO-8859-1',parse_dates=['ocorrencia_dia'],dayfirst=True)

In [ ]:
df1.shape

(6769, 22)

##Enviando ao Mongo

In [ ]:
db3 = client['tarefa']
colecaocenipabruto = db3['Cenipa-brutos']
colecaocenipabruto.count_documents({})

0

In [ ]:
df_dict = df.to_dict("records")
colecaocenipabruto.insert_many(df_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaocenipabruto.count_documents({})

6769

##Trazendo de volta para o df

In [ ]:
# Trazer o dado tratado no colab de volta
cursor = colecaocenipabruto.find({})
dfcenipabruto = pd.DataFrame(list(cursor))

In [ ]:
dfcenipabruto

##Tratando o df Cenipas-Bruto enviado ao Mongo

In [ ]:
# Drop de colunas identicas ( codigo_ocorrencia1 , codigo_ocorrencia2,codigo_ocorrencia3 , codigo_ocorrencia4)
dfcenipabruto.drop(['codigo_ocorrencia1','codigo_ocorrencia2','codigo_ocorrencia3','codigo_ocorrencia4'],axis=1,inplace=True)
# Drop de colunas com problemas e/ou não utilizadas na analise
dfcenipabruto.drop(['ocorrencia_latitude','ocorrencia_longitude','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao'],axis=1,inplace=True)

In [ ]:
#Renomeando colunas
dfcenipabruto.rename(columns={'ocorrencia_classificacao':'classificacao','ocorrencia_cidade':'cidade','ocorrencia_uf':'uf','ocorrencia_pais':'pais'},inplace=True)
dfcenipabruto.rename(columns={'ocorrencia_aerodromo':'aerodromo','ocorrencia_dia':'data','ocorrencia_hora':'horario','total_aeronaves_envolvidas':'n_aeronaves','ocorrencia_saida_pista':'saida_pista'},inplace=True)

In [ ]:
dfcenipabruto.replace(['***','**NI'],pd.NA,inplace=True)

In [ ]:
dfcenipabruto.replace(pd.NA,np.NaN,inplace=True)

In [ ]:
#Verificar o tamanho
dfcenipabruto.shape

(6769, 19)

##Cenipas Vira coleção Cenipas-Tratados

In [ ]:
db3 = client['tarefa']
colecaocenipatratado = db3['Cenipa-Tratados']
colecaocenipatratado.count_documents({})

0

In [ ]:
dfcenipabruto_dict = dfcenipabruto.to_dict("records")
colecaocenipatratado.insert_many(df_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaocenipatratado.count_documents({})

6769

# 2º Df - Alesp - depsp.csv

##Incluindo o arquivo CSV

In [ ]:
path = '/content/depsp.csv'

In [ ]:
## Ler o arquivo csv
df2 = pd.read_csv(path,sep=',',encoding='ISO-8859-1')

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df2.shape

(542395, 8)

In [ ]:
df2

##Enviando ao Mongo

In [ ]:
db3 = client['tarefa']
colecaoalespbruto = db3['Alesp-brutos']
colecaoalespbruto.count_documents({})

0

In [ ]:
df2_dict = df2.to_dict("records")
colecaoalespbruto.insert_many(df2_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaoalespbruto.count_documents({})

542395

##Trazendo de volta para o df

In [ ]:
# Trazer o dado tratado no colab de volta
cursor = colecaoalespbruto.find({})
dfcalespbruto = pd.DataFrame(list(cursor))

In [ ]:
dfcalespbruto

,_id,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,640a42320de7247aecee0b34,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÃOS MARV LTDA
1,640a42320de7247aecee0b35,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÃRIO E OUTROS MATERIAI...,PAPER FACE COMERCIAL LTDA
2,640a42320de7247aecee0b36,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,640a42320de7247aecee0b37,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,640a42320de7247aecee0b38,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÃOS MARV LTDA
...,...,...,...,...,...,...,...,...,...
642390,640a438f0de7247aecf7d88a,2010,300446,11,950.00,3001857820.0,CARLOS NEDER,B - LOCAÃÃO E MANUT DE BENS MÃVEIS E IMÃVE...,JOSÃ LUIZDE ANDRADE FEGUEIRA
642391,640a438f0de7247aecf7d88b,2010,300446,11,573.94,62268800000145.0,CARLOS NEDER,B - LOCAÃÃO E MANUT DE BENS MÃVEIS E IMÃVE...,COND.EDIF.VIADUTO JACAREI
642392,640a438f0de7247aecf7d88c,2010,300446,11,524.84,43283811004309.0,CARLOS NEDER,E - MATERIAIS DE ESCRITÃRIO E OUTROS MATERIAI...,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA
642393,640a438f0de7247aecf7d88d,2010,300446,11,6500.00,2447863000137.0,CARLOS NEDER,F - SERVIÃOS TÃCNICOS PROFISSIONAIS (CONSULT...,AMARAL DOMINGOS NAKAZATO E SARTORI ADVOGADOS


In [ ]:
df = dfcalespbruto

In [ ]:
df

,_id,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,640a42320de7247aecee0b34,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÃOS MARV LTDA
1,640a42320de7247aecee0b35,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÃRIO E OUTROS MATERIAI...,PAPER FACE COMERCIAL LTDA
2,640a42320de7247aecee0b36,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,640a42320de7247aecee0b37,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,640a42320de7247aecee0b38,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÃVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÃOS MARV LTDA
...,...,...,...,...,...,...,...,...,...
642390,640a438f0de7247aecf7d88a,2010,300446,11,950.00,3001857820.0,CARLOS NEDER,B - LOCAÃÃO E MANUT DE BENS MÃVEIS E IMÃVE...,JOSÃ LUIZDE ANDRADE FEGUEIRA
642391,640a438f0de7247aecf7d88b,2010,300446,11,573.94,62268800000145.0,CARLOS NEDER,B - LOCAÃÃO E MANUT DE BENS MÃVEIS E IMÃVE...,COND.EDIF.VIADUTO JACAREI
642392,640a438f0de7247aecf7d88c,2010,300446,11,524.84,43283811004309.0,CARLOS NEDER,E - MATERIAIS DE ESCRITÃRIO E OUTROS MATERIAI...,KALUNGA COMERCIO E INDUSTRIA GRAFICA LTDA
642393,640a438f0de7247aecf7d88d,2010,300446,11,6500.00,2447863000137.0,CARLOS NEDER,F - SERVIÃOS TÃCNICOS PROFISSIONAIS (CONSULT...,AMARAL DOMINGOS NAKAZATO E SARTORI ADVOGADOS


## Tratando dados ALesp

In [ ]:
df.loc[df.Tipo == 'A - COMBUSTÃ\x8dVEIS E LUBRIFICANTES',['Tipo']] = 'A - COMBUSTIVEIS E LUBRIFICANTES'
df.loc[df.Tipo == 'B - LOCAÃ\x87Ã\x83O E MANUT DE BENS MÃ\x93VEIS E IMÃ\x93VEIS, CONDOMÃ\x8dNIOS E OUTROS',['Tipo']] = 'B - LOCAÇÃO E MANUTENÇÃO DE MÓVEIS E IMÓVEIS, CONDOMÍNIOS E OUTROS'
df.loc[df.Tipo == 'C - MATERIAIS E SERVIÃ\x87OS DE MANUT E CONSERV DE VEÃ\x8dCULOS ; PEDÃ\x81GIOS',['Tipo']] = 'C - MATERIAIS E SERVIÇOS DE MANUTENÇÃO E CONSERVAÇÃO DE VEÍCULOS E PEDÁGIOS'
df.loc[df.Tipo == 'D - MATERIAIS E SERVIÃ\x87OS GRÃ\x81FICOS, DE CÃ\x93PIAS  E REPRODUÃ\x87Ã\x83O DE DOCS',['Tipo']] = 'D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS E REPRODUÇÃO DE DOCS'
df.loc[df.Tipo == 'E - MATERIAIS DE ESCRITÃ\x93RIO E OUTROS MATERIAIS DE CONSUMO',['Tipo']] = 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO'
df.loc[df.Tipo == 'F - SERVIÃ\x87OS TÃ\x89CNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)',['Tipo']] = 'F - SERVIÕS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC'
df.loc[df.Tipo == 'G - ASSINATURAS DE PERIÃ\x93DICOS, PUBLICAÃ\x87Ã\x95ES, INTERNET E SOFTWARES',['Tipo']] = 'G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, INTERNET E SOFTWARES'
df.loc[df.Tipo == 'H - SERV.UTIL.PÃ\x9aBLICA (TELEF.MÃ\x93VEL/FIXA, ENERGIA, Ã\x81GUA, GÃ\x81S ETC)',['Tipo']] = 'H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXO, ENERGIA, ÁGUA, GÁS E INTERNET'
df.loc[df.Tipo == 'I - HOSPEDAGEM, ALIMENTAÃ\x87Ã\x83O E DESPESAS DE LOCOMOÃ\x87Ã\x83O',['Tipo']] = 'I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO'
df.loc[df.Tipo == 'J - SERVIÃ\x87OS DE COMUNICAÃ\x87Ã\x83O',['Tipo']] = 'J - SERVIÇOS DE COMUNICAÇÃO'
df.loc[df.Tipo == 'K - LOCAÃ\x87Ã\x83O DE BENS MÃ\x93VEIS',['Tipo']] = 'K - LOCAÇÃO DE BENS MÓVEIS'
df.loc[df.Tipo == 'L - LOCAÃ\x87Ã\x83O DE BENS IMÃ\x93VEIS',['Tipo']] = 'L - LOCAÇÃO DE BENS IMÓVEIS'
df.loc[df.Tipo == 'M - MANUTENÃ\x87Ã\x83O DE BENS MÃ\x93VEIS,  IMÃ\x93VEIS, CONDOMÃ\x8dNIOS E OUTROS',['Tipo']] = 'M - MANUTENÇAÕ DE BENS MÓVEIS, IMÓVEIS, CONDOMÍNIOS E OUTROS'
df.loc[df.Tipo == 'P - DIVULGAÃ\x87Ã\x83O DA ATIVIDADE PARLAMENTAR',['Tipo']] = 'P - DIVULGAÇÃO DE ATIVIDADE PARLAMENTAR'
df.loc[df.Tipo == 'O - LOCAÃ\x87Ã\x83O DE VEÃ\x8dCULO',['Tipo']] = 'O - LOCAÇÃO DE VEÍCULOS'
df.loc[df.Tipo == 'F - SERVIÕS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC',['Tipo']] = 'F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC'



In [ ]:
dfcalespbruto = df

In [ ]:
dfcalespbruto

In [ ]:
correcoes = {
    'VIVO':'VIVO S.A.',
    'TELEFONICA BRASIL S.A':'TELEFONICA BRASIL S.A.',
    'NEXTEL TELECOMUNICAÇÕES LTDA':'NEXTEL TELECOMUNICAÇÕES LTDA.',
    'SABESP':'COMPANHIA DE SANEAMENTO BÁSICO DO ESTADO DE SÃO PAULO - SABESP',
    'TELEFÔNICA BRASIL S.A.':'TELEFONICA BRASIL S.A.',
    'TELEFONICA': 'TELEFONICA BRASIL S.A.',
    'COMPANHIA PAULISTA DE FORÇA E LUZ':'CPFL ENERGIA',
    'TELEFONICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL SA':'TELEFONICA BRASIL S.A.',
    'CLARO S.A.':'CLARO S/A',
    'ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.':'ELETROPAULO',
    'ELETROPAULO':'AES ELETROPAULO',
    'CLARO S.A':'CLARO S/A',
    'NEXTEL TELECOMUNICAÇÕES LTDA.':'NEXTEL TELECOMUNICAÇÕES LTDA',
    'NET':'NET SERVIÇOS DE COMUNICAÇÃO S/A',
    'TELEFÔNICA BRASIL S.A':'TELEFONICA BRASIL S.A.',
    'KALUNGA':'KALUNGA COM. E IND. GRÁFICA LTDA.',
    'KALUNGA COMERCIO E IND. GRAFICA LTDA':'KALUNGA COM. E IND. GRÁFICA LTDA.',
    'NEXTEL':'NEXTEL TELECOMUNICAÇÕES LTDA',
    'KALUNGA COM. E IND. GRAFICA LTDA':'KALUNGA COM. E IND. GRÁFICA LTDA.',
    'TELEFÔNICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELESP CELULAR':'TELESP CELULAR S/A',
    'TELEFÃNICA BRASIL S.A.':'TELEFONICA BRASIL S.A.',
    'TELEFÃNICA BRASIL S.A': 'TELEFONICA BRASIL S.A.',
    'TELEFÃNICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELEFÃNICA':'TELEFONICA BRASIL S.A.',
    'TELEFÃNICA BRASIL S.A':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL  S/A':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL S/A':'TELEFONICA BRASIL S.A.',
    'TELEFONICA BRASIL S/A.':'TELEFONICA BRASIL S.A.'
}

for key, value in correcoes.items():
  dfcalespbruto.loc[dfcalespbruto.Fornecedor==key,'Fornecedor'] = value




In [ ]:
dfcalespbruto

##Alesp Vira coleção Alesp-Tratados

In [ ]:
db3 = client['tarefa']
colecaoalesptratado = db3['Alesp-Tratados']
colecaoalesptratado.count_documents({})

0

In [ ]:
dfcalespbruto_dict = dfcalespbruto.to_dict("records")
colecaoalesptratado.insert_many(dfcalespbruto_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaoalesptratado.count_documents({})

6769

# 3º Df - AirBnB - Amsterdam.csv

##Incluindo o arquivo CSV

In [ ]:
path = '/content/amsterdam_weekdays.csv'

In [ ]:
## Ler o arquivo csv
df3 = pd.read_csv(path,sep=',',encoding='ISO-8859-1')

In [ ]:
df3.shape

(1103, 20)

In [ ]:
df3

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,1099,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,1100,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


##Enviando ao Mongo

In [ ]:
db3 = client['tarefa']
colecaoairbnbbruto = db3['AirBnB-brutos']
colecaoairbnbbruto.count_documents({})

0

In [ ]:
# Transforma em dicionario com listas
df3_dict = df3.to_dict("records")
colecaoairbnbbruto.insert_many(df3_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaoairbnbbruto.count_documents({})

1103

##Trazendo de volta para o df

In [ ]:
# Trazer o dado tratado no colab de volta
cursor = colecaoairbnbbruto.find({})
dfairbnbbruto = pd.DataFrame(list(cursor))

In [ ]:
dfairbnbbruto

,_id,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,...,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,640a6b80d7ce83fa60f5a491,0,194.033698,Private room,False,True,2.0,False,1,0,...,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,640a6b80d7ce83fa60f5a492,1,344.245776,Private room,False,True,4.0,False,0,0,...,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,640a6b80d7ce83fa60f5a493,2,264.101422,Private room,False,True,2.0,False,0,1,...,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,640a6b80d7ce83fa60f5a494,3,433.529398,Private room,False,True,4.0,False,0,1,...,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,640a6b80d7ce83fa60f5a495,4,485.552926,Private room,False,True,2.0,True,0,0,...,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,640a6b80d7ce83fa60f5a8db,1098,2486.115342,Entire home/apt,False,False,2.0,False,0,0,...,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,640a6b80d7ce83fa60f5a8dc,1099,233.637194,Private room,False,True,2.0,False,1,0,...,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,640a6b80d7ce83fa60f5a8dd,1100,317.062311,Private room,False,True,2.0,False,1,0,...,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,640a6b80d7ce83fa60f5a8de,1101,1812.855904,Entire home/apt,False,False,4.0,False,0,0,...,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


## Tratando AirBnB

In [ ]:
dfairbnbbruto.drop(['Unnamed: 0', ],axis=1,inplace=True)

In [ ]:
dfairbnbbruto.rename(columns={'cleanliness_rating':'Classificação_da_Limpeza', 'guest_satisfaction_overall':'Satisfação_Geral_do_Hóspede', 'bedrooms':'Qtde_Quartos', 'dist':'Distância_do_Centro_da_Cidade', 'realSum':'Preço_Total', 'metro_dist':'Distancia_Metro_mais_próximo'}, inplace=True)

In [ ]:
#Renomeando colunas #Renomeando colunas df.rename(columns={'nomeatual':'nomenovo', 'nomeatual1':'nomenovo1'}, inplace=True)
dfairbnbbruto.rename(columns={'room_type':'Tipo_de_Quarto', 'room_shared':'Quarto_Compartilhado', 'room_private':'Quarto_Privativo','person_capacity':'Capacidade_Máxima_Pessoas','host_is_superhost':'Host_é_SuperHost'},inplace=True)


In [ ]:
dfairbnbbruto

,_id,Preço_Total,Tipo_de_Quarto,Quarto_Compartilhado,Quarto_Privativo,Capacidade_Máxima_Pessoas,Host_é_SuperHost,multi,biz,Classificação_da_Limpeza,Satisfação_Geral_do_Hóspede,Qtde_Quartos,Distância_do_Centro_da_Cidade,Distancia_Metro_mais_próximo,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,640a6b80d7ce83fa60f5a491,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,640a6b80d7ce83fa60f5a492,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,640a6b80d7ce83fa60f5a493,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,640a6b80d7ce83fa60f5a494,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,640a6b80d7ce83fa60f5a495,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,640a6b80d7ce83fa60f5a8db,2486.115342,Entire home/apt,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,640a6b80d7ce83fa60f5a8dc,233.637194,Private room,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,640a6b80d7ce83fa60f5a8dd,317.062311,Private room,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,640a6b80d7ce83fa60f5a8de,1812.855904,Entire home/apt,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


In [ ]:
correcoes = {
    'Entire home/apt':'Casa/Apt Inteiro',
    'Private room':'Quarto Privativo',
    'Shared room':'Quarto Compartilhado',
}

for key, value in correcoes.items():
  dfairbnbbruto.loc[dfairbnbbruto.Tipo_de_Quarto==key,'Tipo_de_Quarto'] = value

##AirBnB Vira coleção AirBnB-Tratados

In [ ]:
db3 = client['tarefa']
colecaoairbnbtratado = db3['AirBnB-Tratados']
colecaoairbnbtratado.count_documents({})

0

In [ ]:
dfairbnbbruto_dict = dfairbnbbruto.to_dict("records")
colecaoairbnbtratado.insert_many(dfairbnbbruto_dict)

In [ ]:
# Contagem de linhas deve ter sempre o numero de linhas do shape tem que ser exato validação
colecaoairbnbtratado.count_documents({})

1103